# A물류센터 (purchase & sales)

In [18]:
# Combined Purchase & Sales Data Preprocessing Pipeline
# Integrates best practices from both preprocessing approaches

import pandas as pd
import numpy as np
import re
import unicodedata
from collections import Counter
from difflib import get_close_matches
import warnings
warnings.filterwarnings(action='ignore')

# ============================================================================
# SECTION 1: CLASSIFICATION KEY CREATION (from File 1 - more sophisticated)
# ============================================================================

def make_main_key(name):
    """
    Creates a standardized classification key by cleaning product names.
    Removes numbers, units, special characters, and brackets.
    """
    if pd.isna(name):
        return None

    s = unicodedata.normalize("NFKC", str(name)).lower()
    # Remove content in brackets
    s = re.sub(r"[\(\[\{<].*?[\)\]\}>]", "", s)
    # Remove number patterns with units
    s = re.sub(r"\b\d+(?:\.\d+)?\s*([x×*]\s*\d+)\s*(ea|cs|set|팩|개|입)?", "", s)
    # Remove units
    unit = r"(ml|mL|l|ℓ|g|kg|mg|oz|lb|mm|cm|m|ea|pcs?|pack|box|set|sheet|roll|포|봉|캔|병|팩|개|입|매|매입|세트)"
    s = re.sub(rf"\b\d+(?:\.\d+)?\s*{unit}\b", "", s, flags=re.IGNORECASE)
    # Remove remaining numbers
    s = re.sub(r"\b\d+(?:\.\d+)?", "", s)
    # Remove special characters
    s = re.sub(r"[\[\]<>＜＞/&+/×*_\-~·•:;|,.!?\^\"\'\`]", "", s)
    # Remove extra spaces
    s = re.sub(r"\s+", "", s).strip()

    return s if s else None


def split_container_key(df, key_col='분류키'):
    """
    Identifies container products and adds '_용기' suffix to their classification key.
    Uses strict patterns and food exclusion rules.
    """
    # Strict container patterns
    container_strict_pat = r"(밀폐용기|보관용기|반찬통|찬합|도시락통|락앤락|트라이탄|보관탑|보관병|보온도시락|보온병|보냉백|쿨러백|푸드컨테이너|지퍼백|지퍼락|트레이|보관함|유리밀폐|스텐밀폐|스테인리스용기|김치통|양념통|분유컨테이너|수저통|양념병)"

    # Food patterns to exclude
    food_pat = r"(껌|젤리톨|캔디|사탕|초콜릿|초콜|비스킷|쿠키|젤리|카라멜|츄잉|젤|스낵|라면|우동|국수|면|과자|음료|커피|차|시리얼|누룽지)"

    # Match strict container patterns
    m_strict = df["상품명"].str.contains(container_strict_pat, regex=True, na=False)

    # Match loose container pattern (contains "용기") but not food
    m_loose = df["상품명"].str.contains(r"용기", regex=True, na=False) & ~df["상품명"].str.contains(food_pat, regex=True, na=False)

    # Combine matches and exclude food
    mask_container = (m_strict | m_loose) & ~df["상품명"].str.contains(food_pat, regex=True, na=False)

    # Add '_용기' suffix
    df.loc[mask_container, key_col] = df.loc[mask_container, key_col] + "_용기"

    return df


# ============================================================================
# SECTION 2: AUTOMATED CLASSIFICATION FILLING (from File 1)
# ============================================================================

def fill_main_by_key(df, key_col, label_col, min_support=3, purity=0.9,
                     restrict_to_existing=True, fallback_value=None):
    """
    Automatically fills missing classifications based on key patterns.
    Uses statistical validation with minimum support and purity thresholds.
    """
    out = df.copy()

    # Calculate frequency of (key, label) pairs
    g = (out.dropna(subset=[key_col, label_col])
            .groupby([key_col, label_col]).size()
            .rename('cnt').reset_index())

    if g.empty:
        return out, {}, pd.DataFrame()

    # Calculate total count per key and most frequent label
    tot = g.groupby(key_col)['cnt'].sum().rename('tot')
    top = (g.sort_values(['cnt'], ascending=False)
                .groupby(key_col)
                .head(1)
                .set_index(key_col))

    stat = top.join(tot).reset_index()
    stat['purity'] = stat['cnt'] / stat['tot']

    # Filter by minimum support and purity
    ok = stat[(stat['tot'] >= min_support) & (stat['purity'] >= purity)]
    key2label = dict(zip(ok[key_col], ok[label_col]))

    # Restrict to existing classification values
    if restrict_to_existing:
        candidates = df[label_col].dropna().unique().tolist()
        new_key2label = {}

        for k, v in key2label.items():
            if v in candidates:
                new_key2label[k] = v
            else:
                # Try fuzzy matching
                match = get_close_matches(v, candidates, n=1, cutoff=0.6)
                if match:
                    new_key2label[k] = match[0]
                elif fallback_value is not None:
                    new_key2label[k] = fallback_value

        key2label = new_key2label

    # Fill missing values
    mask = out[label_col].isna()
    out.loc[mask, label_col] = out.loc[mask, key_col].map(key2label)

    # Create statistics report
    report = (stat.assign(adopted=stat[key_col].isin(ok[key_col]))
                    .sort_values(['adopted', 'purity', 'tot'],
                                ascending=[True, False, False]))

    return out, key2label, report


# ============================================================================
# SECTION 3: BRAND PREFIX ANALYSIS (from File 2)
# ============================================================================

def analyze_brand_prefixes(df, subcol='소분류'):
    """
    Analyzes brand prefixes (text before ']') in product names.
    Returns statistics on how prefixes map to subcategories.
    """
    pat = re.compile(r'^\s*([^]]+)]')

    # Extract prefixes from non-null subcategories
    with_sub = df[df[subcol].notna()].copy()
    prefixes = set()

    for name in with_sub['상품명'].dropna():
        m = pat.match(name)
        if m:
            prefixes.add(m.group(1).strip())

    # Analyze distribution
    stats = []
    for prefix in sorted(prefixes):
        regex = re.compile(rf'(^|\s){re.escape(prefix)}\]')
        matched = with_sub[with_sub['상품명'].str.contains(regex, na=False)]

        if not matched.empty:
            counts = matched[subcol].value_counts()
            for cat, cnt in counts.items():
                stats.append({
                    '브랜드': prefix,
                    '소분류': cat,
                    '상품명_개수': cnt
                })

    return pd.DataFrame(stats).sort_values(['브랜드', '상품명_개수'],
                                          ascending=[True, False])


# ============================================================================
# SECTION 4: SALES DATA HARMONIZATION
# ============================================================================

def harmonize_sales_columns(df, dataset_type='2123'):
    """
    Harmonizes column names between 2021-23 and 2024 sales datasets.

    Parameters:
    -----------
    df : DataFrame
        Sales dataframe to harmonize
    dataset_type : str
        '2123' for 2021-23 data, '24' for 2024 data
    """
    df = df.copy()

    if dataset_type == '2123':
        # Rename columns to match standard naming
        rename_map = {
            '매출처 우편번호': '우편번호',
            '바코드': '상품바코드',
            '공급금액': '공급가액',
            '부가세(과세)': '부가세',
            '옵션 코드': '옵션코드'
        }
        df = df.rename(columns=rename_map)

    elif dataset_type == '24':
        # Rename columns to match standard naming
        rename_map = {
            '상품 바코드(대한상의)': '상품바코드'
        }
        df = df.rename(columns=rename_map)

    return df


def load_and_merge_sales_data(filepath_2123, filepath_24):
    """
    Loads both sales datasets and merges them into a single dataframe.

    Parameters:
    -----------
    filepath_2123 : str
        Path to 2021-23 sales data CSV
    filepath_24 : str
        Path to 2024 sales data CSV
    """
    print("\n[Loading Sales Data]")
    print("-" * 80)

    # Load 2021-23 data
    print(f"Loading {filepath_2123}...")
    df_2123 = pd.read_csv(filepath_2123, dtype=str)
    df_2123 = harmonize_sales_columns(df_2123, dataset_type='2123')
    print(f"   Rows: {len(df_2123):,}")

    # Load 2024 data
    print(f"Loading {filepath_24}...")
    df_24 = pd.read_csv(filepath_24, dtype=str)
    df_24 = harmonize_sales_columns(df_24, dataset_type='24')
    print(f"   Rows: {len(df_24):,}")

    # Merge datasets
    print("\nMerging datasets...")
    df_merged = pd.concat([df_2123, df_24], ignore_index=True)
    print(f"   Total rows after merge: {len(df_merged):,}")

    return df_merged


# ============================================================================
# SECTION 5: MAIN PREPROCESSING PIPELINES
# ============================================================================

def preprocess_purchase_data(filepath, target_categories=None):
    """
    Main preprocessing pipeline for purchase data.

    Parameters:
    -----------
    filepath : str
        Path to the purchase data CSV file
    target_categories : list, optional
        List of target 중분류 categories to filter
        Default: ['생수,음료,건강', '신선식품']
    """

    if target_categories is None:
        target_categories = ['생수,음료,건강', '신선식품']

    print("=" * 80)
    print("PURCHASE DATA PREPROCESSING PIPELINE")
    print("=" * 80)

    # Step 1: Load data
    print("\n[Step 1] Loading data...")
    df = pd.read_csv(filepath, dtype=str)
    print(f"   Initial rows: {len(df):,}")

    # Step 2: Remove returns
    print("\n[Step 2] Removing return entries (반출)...")
    df = df[df['작업유형'] != '반출'].copy()
    print(f"   Rows after removal: {len(df):,}")

    # Step 3: Create classification key
    print("\n[Step 3] Creating classification keys...")
    df['분류키'] = df['상품명'].apply(make_main_key)
    df = split_container_key(df, key_col='분류키')
    print(f"   Unique keys created: {df['분류키'].nunique():,}")

    # Step 4: Fill missing classifications (initial pass - groupby)
    print("\n[Step 4] Initial classification filling (groupby method)...")
    for col in ["대분류", "중분류", "소분류"]:
        before = df[col].isna().sum()
        mask = df[col].isna()
        df.loc[mask, col] = (
            df.groupby("분류키")[col]
            .transform(lambda x: x.fillna(x.dropna().iloc[0]) if x.dropna().size > 0 else x)
        )[mask]
        after = df[col].isna().sum()
        print(f"   {col}: {before:,} → {after:,} missing ({before-after:,} filled)")

    # Step 5: Fill missing barcodes
    print("\n[Step 5] Filling missing barcodes...")
    if "바코드" in df.columns:
        before = df["바코드"].isna().sum()
        mask = df["바코드"].isna()
        if "상품코드" in df.columns:
            df.loc[mask, "바코드"] = (
                df.groupby("상품코드")["바코드"]
                .transform(lambda x: x.fillna(x.dropna().iloc[0]) if x.dropna().size > 0 else x)
            )[mask]
        after = df["바코드"].isna().sum()
        print(f"   Barcodes: {before:,} → {after:,} missing ({before-after:,} filled)")

    # Step 6: Filter by target categories
    print(f"\n[Step 6] Filtering by target categories: {target_categories}...")
    df_filtered = df[df['중분류'].isin(target_categories)].copy()
    print(f"   Rows after filtering: {len(df_filtered):,}")
    print(f"   Unique products: {df_filtered['상품명'].nunique():,}")

    # Step 7: Automated classification filling (advanced)
    print("\n[Step 7] Advanced classification filling...")
    df_updated = df_filtered.copy()

    # Fill 대분류
    before = df_updated['대분류'].isna().sum()
    df_filled, key2label, report = fill_main_by_key(
        df_updated,
        key_col='분류키',
        label_col='대분류',
        min_support=2,
        purity=0.7
    )

    fill_map = df_filled.dropna(subset=['대분류']).groupby('분류키')['대분류'].first()
    mask = df_updated['대분류'].isna()
    df_updated.loc[mask, '대분류'] = df_updated.loc[mask, '분류키'].map(fill_map)
    after = df_updated['대분류'].isna().sum()
    print(f"   대분류: {before:,} → {after:,} missing ({before-after:,} filled)")

    # Step 8: Analyze and handle unclassified data
    print("\n[Step 8] Analyzing unclassified data...")
    unclassified = df_updated[df_updated['소분류'].isna()]
    print(f"   Unclassified rows: {len(unclassified):,}")

    if len(unclassified) > 0:
        # Fill remaining with default category
        default_category = '냉동,냉장,간편식'
        mask = df_updated['소분류'].isna()
        df_updated.loc[mask, '소분류'] = default_category
        print(f"   Filled remaining with default: '{default_category}'")

    # Step 9: Clean up columns
    print("\n[Step 9] Cleaning up columns...")
    columns_to_drop = [
        '작업유형', '매출처코드', '매출처 우편번호', '입고 형태',
        '바코드', '규격', '입수', '수량', '판매금액', '부가세(과세)',
        '상품코드', '옵션 코드', '옵션', '대분류'
    ]

    # Only drop columns that exist
    columns_to_drop = [col for col in columns_to_drop if col in df_updated.columns]
    df_updated = df_updated.drop(columns_to_drop, axis=1)
    print(f"   Dropped {len(columns_to_drop)} columns")
    print(f"   Remaining columns: {list(df_updated.columns)}")

    # Step 10: Final cleanup
    print("\n[Step 10] Final cleanup...")
    df_updated = df_updated.reset_index(drop=True)

    # Summary statistics
    print("\n" + "=" * 80)
    print("PREPROCESSING COMPLETE")
    print("=" * 80)
    print(f"\nFinal dataset shape: {df_updated.shape}")
    print(f"Rows: {len(df_updated):,}")
    print(f"Columns: {len(df_updated.columns)}")
    print(f"\nMissing values per column:")
    print(df_updated.isnull().sum())
    print(f"\nUnique values per classification:")
    print(f"  중분류: {df_updated['중분류'].nunique()}")
    print(f"  소분류: {df_updated['소분류'].nunique()}")

    return df_updated


def preprocess_sales_data(filepath_2123, filepath_24, target_categories=None):
    """
    Main preprocessing pipeline for sales data (combines 2021-23 and 2024 data).

    Parameters:
    -----------
    filepath_2123 : str
        Path to the 2021-23 sales data CSV file
    filepath_24 : str
        Path to the 2024 sales data CSV file
    target_categories : list, optional
        List of target 중분류 categories to filter
        Default: ['생수,음료,건강', '신선식품']
    """

    if target_categories is None:
        target_categories = ['생수,음료,건강', '신선식품']

    print("=" * 80)
    print("SALES DATA PREPROCESSING PIPELINE")
    print("=" * 80)

    # Step 1: Load and merge data
    df = load_and_merge_sales_data(filepath_2123, filepath_24)

    # Step 2: Create classification key
    print("\n[Step 2] Creating classification keys...")
    df['분류키'] = df['상품명'].apply(make_main_key)
    df = split_container_key(df, key_col='분류키')
    print(f"   Unique keys created: {df['분류키'].nunique():,}")

    # Step 3: Fill missing classifications (initial pass - groupby)
    print("\n[Step 3] Initial classification filling (groupby method)...")
    for col in ["대분류", "중분류", "소분류"]:
        before = df[col].isna().sum()
        mask = df[col].isna()
        df.loc[mask, col] = (
            df.groupby("분류키")[col]
            .transform(lambda x: x.fillna(x.dropna().iloc[0]) if x.dropna().size > 0 else x)
        )[mask]
        after = df[col].isna().sum()
        print(f"   {col}: {before:,} → {after:,} missing ({before-after:,} filled)")

    # Step 4: Fill missing barcodes
    print("\n[Step 4] Filling missing barcodes...")
    if "상품바코드" in df.columns:
        before = df["상품바코드"].isna().sum()
        mask = df["상품바코드"].isna()
        df.loc[mask, "상품바코드"] = (
            df.groupby("상품명")["상품바코드"]
            .transform(lambda x: x.fillna(x.dropna().iloc[0]) if x.dropna().size > 0 else x)
        )[mask]
        after = df["상품바코드"].isna().sum()
        print(f"   Barcodes: {before:,} → {after:,} missing ({before-after:,} filled)")

    # Step 5: Filter by target categories
    print(f"\n[Step 5] Filtering by target categories: {target_categories}...")
    df_filtered = df[df['중분류'].isin(target_categories)].copy()
    print(f"   Rows after filtering: {len(df_filtered):,}")
    print(f"   Unique products: {df_filtered['상품명'].nunique():,}")

    # Step 6: Automated classification filling (advanced)
    print("\n[Step 6] Advanced classification filling...")
    df_updated = df_filtered.copy()

    # Fill 대분류
    before = df_updated['대분류'].isna().sum()
    df_filled, key2label, report = fill_main_by_key(
        df_updated,
        key_col='분류키',
        label_col='대분류',
        min_support=2,
        purity=0.7
    )

    fill_map = df_filled.dropna(subset=['대분류']).groupby('분류키')['대분류'].first()
    mask = df_updated['대분류'].isna()
    df_updated.loc[mask, '대분류'] = df_updated.loc[mask, '분류키'].map(fill_map)
    after = df_updated['대분류'].isna().sum()
    print(f"   대분류: {before:,} → {after:,} missing ({before-after:,} filled)")

    # Step 7: Analyze and handle unclassified data
    print("\n[Step 7] Analyzing unclassified data...")
    unclassified = df_updated[df_updated['소분류'].isna()]
    print(f"   Unclassified rows: {len(unclassified):,}")

    if len(unclassified) > 0:
        # Fill remaining with default category
        default_category = '냉동,냉장,간편식'
        mask = df_updated['소분류'].isna()
        df_updated.loc[mask, '소분류'] = default_category
        print(f"   Filled remaining with default: '{default_category}'")

    # Step 8: Clean up columns
    print("\n[Step 8] Cleaning up columns...")
    columns_to_drop = [
        '구분', '매출처코드', '우편번호', '옵션코드',
        '규격', '입수', '상품바코드', '대분류'
    ]

    # Only drop columns that exist
    columns_to_drop = [col for col in columns_to_drop if col in df_updated.columns]
    df_updated = df_updated.drop(columns_to_drop, axis=1)
    print(f"   Dropped {len(columns_to_drop)} columns")
    print(f"   Remaining columns: {list(df_updated.columns)}")

    # Step 9: Final cleanup
    print("\n[Step 9] Final cleanup...")
    df_updated = df_updated.reset_index(drop=True)

    # Summary statistics
    print("\n" + "=" * 80)
    print("PREPROCESSING COMPLETE")
    print("=" * 80)
    print(f"\nFinal dataset shape: {df_updated.shape}")
    print(f"Rows: {len(df_updated):,}")
    print(f"Columns: {len(df_updated.columns)}")
    print(f"\nMissing values per column:")
    print(df_updated.isnull().sum())
    print(f"\nUnique values per classification:")
    print(f"  중분류: {df_updated['중분류'].nunique()}")
    print(f"  소분류: {df_updated['소분류'].nunique()}")

    return df_updated


# ============================================================================
# SECTION 6: USAGE EXAMPLE
# ============================================================================

if __name__ == "__main__":

    # ========================================================================
    # Process PURCHASE data
    # ========================================================================
    print("\n" + "🔹" * 40)
    print("PROCESSING PURCHASE DATA")
    print("🔹" * 40)

    a_purchase_processed = preprocess_purchase_data(
        'A_purchase_data.csv',
        target_categories=['생수,음료,건강', '신선식품']
    )

    # Save processed purchase data
    # purchase_output = 'A_purchase_data_processed.csv'
    # a_purchase_processed.to_csv(purchase_output, index=False, encoding='utf-8-sig')
    # print(f"\n✅ Processed purchase data saved to: {purchase_output}")

    print("\nSample of processed purchase data:")
    print(a_purchase_processed.head(10))

    # ========================================================================
    # Process SALES data
    # ========================================================================
    print("\n\n" + "🔹" * 40)
    print("PROCESSING SALES DATA")
    print("🔹" * 40)

    a_sales_processed = preprocess_sales_data(
        'a_sales_2123.csv',
        'a_sales_24.csv',
        target_categories=['생수,음료,건강', '신선식품']
    )

    # Save processed sales data
    # sales_output = 'A_sales_data_processed.csv'
    # a_sales_processed.to_csv(sales_output, index=False, encoding='utf-8-sig')
    # print(f"\n✅ Processed sales data saved to: {sales_output}")

    print("\nSample of processed sales data:")
    print(a_sales_processed.head(10))

    # ========================================================================
    # Final Summary
    # ========================================================================
    print("\n\n" + "=" * 80)
    print("🎉 ALL PREPROCESSING COMPLETE!")
    print("=" * 80)
    print(f"\n📊 Purchase Data: {len(a_purchase_processed):,} rows")
    print(f"📊 Sales Data: {len(a_sales_processed):,} rows")
    print(f"\n💾 Output files:")
    # print(f"   - {purchase_output}")
    # print(f"   - {sales_output}")


🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹
PROCESSING PURCHASE DATA
🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹
PURCHASE DATA PREPROCESSING PIPELINE

[Step 1] Loading data...
   Initial rows: 227,534

[Step 2] Removing return entries (반출)...
   Rows after removal: 201,534

[Step 3] Creating classification keys...
   Unique keys created: 6,170

[Step 4] Initial classification filling (groupby method)...
   대분류: 1,623 → 571 missing (1,052 filled)
   중분류: 1,845 → 690 missing (1,155 filled)
   소분류: 113,776 → 106,776 missing (7,000 filled)

[Step 5] Filling missing barcodes...
   Barcodes: 584 → 525 missing (59 filled)

[Step 6] Filtering by target categories: ['생수,음료,건강', '신선식품']...
   Rows after filtering: 28,295
   Unique products: 1,249

[Step 7] Advanced classification filling...
   대분류: 0 → 0 missing (0 filled)

[Step 8] Analyzing unclassified data...
   Unclassified rows: 20,267
   Filled remaining with default: '냉동,냉장,간편식'

[Step 9] Cleaning up columns...
   Dropped 14 columns
   Remain

In [19]:
categories_unique = a_purchase_processed['중분류'].unique()
print(categories_unique)

categories_unique = a_sales_processed['중분류'].unique()
print(categories_unique)

['생수,음료,건강' '신선식품']
['생수,음료,건강' '신선식품']


In [20]:
print(a_purchase_processed.info())
print('<=============================================>')
print(a_sales_processed.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28295 entries, 0 to 28294
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   일자         28295 non-null  object
 1   공급업체 코드    28295 non-null  object
 2   공급업체 우편번호  28295 non-null  object
 3   상품명        28295 non-null  object
 4   EA         28295 non-null  object
 5   중분류        28295 non-null  object
 6   소분류        28295 non-null  object
 7   분류키        28295 non-null  object
dtypes: object(8)
memory usage: 1.7+ MB
None
<=============================================>
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229074 entries, 0 to 229073
Data columns (total 8 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   판매일     229074 non-null  object
 1   판매수량    229074 non-null  object
 2   상품명     229074 non-null  object
 3   중분류     229074 non-null  object
 4   소분류     229074 non-null  object
 5   공급가액    229074 non-null  object
 6 

# B물류센터 (purchase & sales)

In [25]:
# B물류센터 Combined Purchase & Sales Data Preprocessing Pipeline
# Filters by 소분류 (탄산음료, 봉지라면)

import pandas as pd
import numpy as np
import re
import unicodedata
from collections import Counter
from difflib import get_close_matches
import warnings
warnings.filterwarnings(action='ignore')

# ============================================================================
# SECTION 1: CLASSIFICATION KEY CREATION
# ============================================================================

def make_main_key(name):
    """
    Creates a standardized classification key by cleaning product names.
    Removes numbers, units, special characters, and brackets.
    """
    if pd.isna(name):
        return None

    s = unicodedata.normalize("NFKC", str(name)).lower()
    # Remove content in brackets
    s = re.sub(r"[\(\[\{<].*?[\)\]\}>]", "", s)
    # Remove number patterns with units
    s = re.sub(r"\b\d+(?:\.\d+)?\s*([x×*]\s*\d+)\s*(ea|cs|set|팩|개|입)?", "", s)
    # Remove units
    unit = r"(ml|mL|l|ℓ|g|kg|mg|oz|lb|mm|cm|m|ea|pcs?|pack|box|set|sheet|roll|포|봉|캔|병|팩|개|입|매|매입|세트)"
    s = re.sub(rf"\b\d+(?:\.\d+)?\s*{unit}\b", "", s, flags=re.IGNORECASE)
    # Remove remaining numbers
    s = re.sub(r"\b\d+(?:\.\d+)?", "", s)
    # Remove special characters
    s = re.sub(r"[\[\]<>＜＞/&+/×*_\-~·•:;|,.!?\^\"\'\`]", "", s)
    # Remove extra spaces
    s = re.sub(r"\s+", "", s).strip()

    return s if s else None


def split_container_key(df, key_col='분류키'):
    """
    Identifies container products and adds '_용기' suffix to their classification key.
    Uses strict patterns and food exclusion rules.
    """
    # Strict container patterns
    container_strict_pat = r"(밀폐용기|보관용기|반찬통|찬합|도시락통|락앤락|트라이탄|보관탑|보관병|보온도시락|보온병|보냉백|쿨러백|푸드컨테이너|지퍼백|지퍼락|트레이|보관함|유리밀폐|스텐밀폐|스테인리스용기|김치통|양념통|분유컨테이너|수저통|양념병)"

    # Food patterns to exclude
    food_pat = r"(껌|젤리톨|캔디|사탕|초콜릿|초콜|비스킷|쿠키|젤리|카라멜|츄잉|젤|스낵|라면|우동|국수|면|과자|음료|커피|차|시리얼|누룽지)"

    # Match strict container patterns
    m_strict = df["상품명"].str.contains(container_strict_pat, regex=True, na=False)

    # Match loose container pattern (contains "용기") but not food
    m_loose = df["상품명"].str.contains(r"용기", regex=True, na=False) & ~df["상품명"].str.contains(food_pat, regex=True, na=False)

    # Combine matches and exclude food
    mask_container = (m_strict | m_loose) & ~df["상품명"].str.contains(food_pat, regex=True, na=False)

    # Add '_용기' suffix
    df.loc[mask_container, key_col] = df.loc[mask_container, key_col] + "_용기"

    return df


# ============================================================================
# SECTION 2: AUTOMATED CLASSIFICATION FILLING
# ============================================================================

def fill_main_by_key(df, key_col, label_col, min_support=3, purity=0.9,
                     restrict_to_existing=True, fallback_value=None):
    """
    Automatically fills missing classifications based on key patterns.
    Uses statistical validation with minimum support and purity thresholds.
    """
    out = df.copy()

    # Calculate frequency of (key, label) pairs
    g = (out.dropna(subset=[key_col, label_col])
            .groupby([key_col, label_col]).size()
            .rename('cnt').reset_index())

    if g.empty:
        return out, {}, pd.DataFrame()

    # Calculate total count per key and most frequent label
    tot = g.groupby(key_col)['cnt'].sum().rename('tot')
    top = (g.sort_values(['cnt'], ascending=False)
                .groupby(key_col)
                .head(1)
                .set_index(key_col))

    stat = top.join(tot).reset_index()
    stat['purity'] = stat['cnt'] / stat['tot']

    # Filter by minimum support and purity
    ok = stat[(stat['tot'] >= min_support) & (stat['purity'] >= purity)]
    key2label = dict(zip(ok[key_col], ok[label_col]))

    # Restrict to existing classification values
    if restrict_to_existing:
        candidates = df[label_col].dropna().unique().tolist()
        new_key2label = {}

        for k, v in key2label.items():
            if v in candidates:
                new_key2label[k] = v
            else:
                # Try fuzzy matching
                match = get_close_matches(v, candidates, n=1, cutoff=0.6)
                if match:
                    new_key2label[k] = match[0]
                elif fallback_value is not None:
                    new_key2label[k] = fallback_value

        key2label = new_key2label

    # Fill missing values
    mask = out[label_col].isna()
    out.loc[mask, label_col] = out.loc[mask, key_col].map(key2label)

    # Create statistics report
    report = (stat.assign(adopted=stat[key_col].isin(ok[key_col]))
                    .sort_values(['adopted', 'purity', 'tot'],
                                ascending=[True, False, False]))

    return out, key2label, report


# ============================================================================
# SECTION 3: BRAND PREFIX ANALYSIS
# ============================================================================

def analyze_brand_prefixes(df, subcol='소분류'):
    """
    Analyzes brand prefixes (text before ']') in product names.
    Returns statistics on how prefixes map to subcategories.
    """
    pat = re.compile(r'^\s*([^]]+)]')

    # Extract prefixes from non-null subcategories
    with_sub = df[df[subcol].notna()].copy()
    prefixes = set()

    for name in with_sub['상품명'].dropna():
        m = pat.match(name)
        if m:
            prefixes.add(m.group(1).strip())

    # Analyze distribution
    stats = []
    for prefix in sorted(prefixes):
        regex = re.compile(rf'(^|\s){re.escape(prefix)}\]')
        matched = with_sub[with_sub['상품명'].str.contains(regex, na=False)]

        if not matched.empty:
            counts = matched[subcol].value_counts()
            for cat, cnt in counts.items():
                stats.append({
                    '브랜드': prefix,
                    '소분류': cat,
                    '상품명_개수': cnt
                })

    return pd.DataFrame(stats).sort_values(['브랜드', '상품명_개수'],
                                          ascending=[True, False])


# ============================================================================
# SECTION 4: SALES DATA HARMONIZATION
# ============================================================================

def harmonize_sales_columns(df, dataset_type='2123'):
    """
    Harmonizes column names between 2021-23 and 2024 sales datasets.

    Parameters:
    -----------
    df : DataFrame
        Sales dataframe to harmonize
    dataset_type : str
        '2123' for 2021-23 data, '24' for 2024 data
    """
    df = df.copy()

    # Print current columns for debugging
    print(f"   Columns before harmonization ({dataset_type}):")
    print(f"   {list(df.columns)}")

    if dataset_type == '2123':
        # Rename columns to match standard naming
        rename_map = {
            '매출처 우편번호': '우편번호',
            '바코드': '상품바코드',
            '공급금액': '공급가액',
            '부가세(과세)': '부가세',
            '옵션 코드': '옵션코드'
        }

        # Check which columns actually exist
        existing_renames = {k: v for k, v in rename_map.items() if k in df.columns}
        print(f"   Renaming: {existing_renames}")
        df = df.rename(columns=existing_renames)

    elif dataset_type == '24':
        # Rename columns to match standard naming
        rename_map = {
            '상품 바코드(대한상의)': '상품바코드',
            '옵션코드': '옵션코드'  # Ensure consistency
        }

        # Check which columns actually exist
        existing_renames = {k: v for k, v in rename_map.items() if k in df.columns}
        print(f"   Renaming: {existing_renames}")
        df = df.rename(columns=existing_renames)

    print(f"   Columns after harmonization:")
    print(f"   {list(df.columns)}\n")

    return df


def load_and_merge_sales_data(filepath_2123, filepath_24):
    """
    Loads both sales datasets and merges them into a single dataframe.

    Parameters:
    -----------
    filepath_2123 : str
        Path to 2021-23 sales data CSV
    filepath_24 : str
        Path to 2024 sales data CSV
    """
    print("\n[Loading Sales Data]")
    print("-" * 80)

    # Load 2021-23 data
    print(f"Loading {filepath_2123}...")
    df_2123 = pd.read_csv(filepath_2123, dtype=str)
    print(f"   Rows: {len(df_2123):,}")
    df_2123 = harmonize_sales_columns(df_2123, dataset_type='2123')

    # Load 2024 data
    print(f"Loading {filepath_24}...")
    df_24 = pd.read_csv(filepath_24, dtype=str)
    print(f"   Rows: {len(df_24):,}")
    df_24 = harmonize_sales_columns(df_24, dataset_type='24')

    # Merge datasets
    print("Merging datasets...")
    df_merged = pd.concat([df_2123, df_24], ignore_index=True)
    print(f"   Total rows after merge: {len(df_merged):,}")

    # Consolidate duplicate columns (in case renaming didn't work perfectly)
    print("\nConsolidating duplicate columns...")
    duplicate_pairs = [
        ('공급가액', '공급금액'),
        ('부가세', '부가세(과세)'),
        ('우편번호', '매출처 우편번호'),
        ('상품바코드', '바코드'),
        ('옵션코드', '옵션 코드')
    ]

    for primary, secondary in duplicate_pairs:
        if primary in df_merged.columns and secondary in df_merged.columns:
            print(f"   Merging {primary} ← {secondary}")
            # Fill primary column with secondary where primary is null
            df_merged[primary] = df_merged[primary].fillna(df_merged[secondary])
            # Drop the secondary column
            df_merged = df_merged.drop(columns=[secondary])
        elif secondary in df_merged.columns and primary not in df_merged.columns:
            print(f"   Renaming {secondary} → {primary}")
            df_merged = df_merged.rename(columns={secondary: primary})

    print(f"\n   Final columns after consolidation:")
    print(f"   {list(df_merged.columns)}")
    print(f"   Non-null counts:")
    for col in df_merged.columns:
        print(f"      {col}: {df_merged[col].notna().sum():,}")

    return df_merged


# ============================================================================
# SECTION 5: MAIN PREPROCESSING PIPELINES
# ============================================================================

def preprocess_purchase_data(filepath, target_categories=None):
    """
    Main preprocessing pipeline for B물류센터 purchase data.
    Filters by 소분류 (not 중분류 like A물류센터).

    Parameters:
    -----------
    filepath : str
        Path to the purchase data CSV file
    target_categories : list, optional
        List of target 소분류 categories to filter
        Default: ['탄산음료', '봉지라면']
    """

    if target_categories is None:
        target_categories = ['탄산음료', '봉지라면']

    print("=" * 80)
    print("B물류센터 PURCHASE DATA PREPROCESSING PIPELINE")
    print("=" * 80)

    # Step 1: Load data
    print("\n[Step 1] Loading data...")
    df = pd.read_csv(filepath, dtype=str)
    print(f"   Initial rows: {len(df):,}")

    # Step 2: Remove returns
    print("\n[Step 2] Removing return entries (반출)...")
    df = df[df['작업유형'] != '반출'].copy()
    print(f"   Rows after removal: {len(df):,}")

    # Step 3: Create classification key
    print("\n[Step 3] Creating classification keys...")
    df['분류키'] = df['상품명'].apply(make_main_key)
    df = split_container_key(df, key_col='분류키')
    print(f"   Unique keys created: {df['분류키'].nunique():,}")

    # Step 4: Fill missing classifications (initial pass - groupby)
    print("\n[Step 4] Initial classification filling (groupby method)...")
    for col in ["대분류", "중분류", "소분류"]:
        before = df[col].isna().sum()
        mask = df[col].isna()
        df.loc[mask, col] = (
            df.groupby("분류키")[col]
            .transform(lambda x: x.fillna(x.dropna().iloc[0]) if x.dropna().size > 0 else x)
        )[mask]
        after = df[col].isna().sum()
        print(f"   {col}: {before:,} → {after:,} missing ({before-after:,} filled)")

    # Step 5: Fill missing barcodes
    print("\n[Step 5] Filling missing barcodes...")
    if "바코드" in df.columns:
        before = df["바코드"].isna().sum()
        mask = df["바코드"].isna()
        if "상품코드" in df.columns:
            df.loc[mask, "바코드"] = (
                df.groupby("상품코드")["바코드"]
                .transform(lambda x: x.fillna(x.dropna().iloc[0]) if x.dropna().size > 0 else x)
            )[mask]
        after = df["바코드"].isna().sum()
        print(f"   Barcodes: {before:,} → {after:,} missing ({before-after:,} filled)")

    # Step 6: Filter by target categories (소분류 for B물류센터)
    print(f"\n[Step 6] Filtering by target 소분류: {target_categories}...")
    df_filtered = df[df['소분류'].isin(target_categories)].copy()
    print(f"   Rows after filtering: {len(df_filtered):,}")
    print(f"   Unique products: {df_filtered['상품명'].nunique():,}")

    # Step 7: Automated classification filling (advanced)
    print("\n[Step 7] Advanced classification filling...")
    df_updated = df_filtered.copy()

    # Fill 대분류
    before = df_updated['대분류'].isna().sum()
    df_filled, key2label, report = fill_main_by_key(
        df_updated,
        key_col='분류키',
        label_col='대분류',
        min_support=2,
        purity=0.7
    )

    fill_map = df_filled.dropna(subset=['대분류']).groupby('분류키')['대분류'].first()
    mask = df_updated['대분류'].isna()
    df_updated.loc[mask, '대분류'] = df_updated.loc[mask, '분류키'].map(fill_map)
    after = df_updated['대분류'].isna().sum()
    print(f"   대분류: {before:,} → {after:,} missing ({before-after:,} filled)")

    # Step 8: Clean up columns
    print("\n[Step 8] Cleaning up columns...")
    columns_to_drop = [
        '작업유형', '매출처코드', '매출처 우편번호', '입고 형태',
        '바코드', '규격', '입수', '수량', '판매금액', '부가세(과세)',
        '상품코드', '옵션 코드', '옵션', '대분류'
    ]

    # Only drop columns that exist
    columns_to_drop = [col for col in columns_to_drop if col in df_updated.columns]
    df_updated = df_updated.drop(columns_to_drop, axis=1)
    print(f"   Dropped {len(columns_to_drop)} columns")
    print(f"   Remaining columns: {list(df_updated.columns)}")

    # Step 9: Final cleanup
    print("\n[Step 9] Final cleanup...")
    df_updated = df_updated.reset_index(drop=True)

    # Summary statistics
    print("\n" + "=" * 80)
    print("PREPROCESSING COMPLETE")
    print("=" * 80)
    print(f"\nFinal dataset shape: {df_updated.shape}")
    print(f"Rows: {len(df_updated):,}")
    print(f"Columns: {len(df_updated.columns)}")
    print(f"\nMissing values per column:")
    print(df_updated.isnull().sum())
    print(f"\nUnique values per classification:")
    print(f"  중분류: {df_updated['중분류'].nunique()}")
    print(f"  소분류: {df_updated['소분류'].nunique()}")

    return df_updated


def preprocess_sales_data(filepath_2123, filepath_24, target_categories=None):
    """
    Main preprocessing pipeline for B물류센터 sales data.
    Filters by 소분류 (not 중분류 like A물류센터).

    Parameters:
    -----------
    filepath_2123 : str
        Path to the 2021-23 sales data CSV file
    filepath_24 : str
        Path to the 2024 sales data CSV file
    target_categories : list, optional
        List of target 소분류 categories to filter
        Default: ['탄산음료', '봉지라면']
    """

    if target_categories is None:
        target_categories = ['탄산음료', '봉지라면']

    print("=" * 80)
    print("B물류센터 SALES DATA PREPROCESSING PIPELINE")
    print("=" * 80)

    # Step 1: Load and merge data
    df = load_and_merge_sales_data(filepath_2123, filepath_24)

    # Step 2: Create classification key
    print("\n[Step 2] Creating classification keys...")
    df['분류키'] = df['상품명'].apply(make_main_key)
    df = split_container_key(df, key_col='분류키')
    print(f"   Unique keys created: {df['분류키'].nunique():,}")

    # Step 3: Fill missing classifications (initial pass - groupby)
    print("\n[Step 3] Initial classification filling (groupby method)...")
    for col in ["대분류", "중분류", "소분류"]:
        before = df[col].isna().sum()
        mask = df[col].isna()
        df.loc[mask, col] = (
            df.groupby("분류키")[col]
            .transform(lambda x: x.fillna(x.dropna().iloc[0]) if x.dropna().size > 0 else x)
        )[mask]
        after = df[col].isna().sum()
        print(f"   {col}: {before:,} → {after:,} missing ({before-after:,} filled)")

    # Step 4: Fill missing barcodes
    print("\n[Step 4] Filling missing barcodes...")
    if "상품바코드" in df.columns:
        before = df["상품바코드"].isna().sum()
        mask = df["상품바코드"].isna()
        df.loc[mask, "상품바코드"] = (
            df.groupby("상품명")["상품바코드"]
            .transform(lambda x: x.fillna(x.dropna().iloc[0]) if x.dropna().size > 0 else x)
        )[mask]
        after = df["상품바코드"].isna().sum()
        print(f"   Barcodes: {before:,} → {after:,} missing ({before-after:,} filled)")

    # Step 5: Filter by target categories (소분류 for B물류센터)
    print(f"\n[Step 5] Filtering by target 소분류: {target_categories}...")
    df_filtered = df[df['소분류'].isin(target_categories)].copy()
    print(f"   Rows after filtering: {len(df_filtered):,}")
    print(f"   Unique products: {df_filtered['상품명'].nunique():,}")

    # Step 6: Automated classification filling (advanced)
    print("\n[Step 6] Advanced classification filling...")
    df_updated = df_filtered.copy()

    # Fill 대분류
    before = df_updated['대분류'].isna().sum()
    df_filled, key2label, report = fill_main_by_key(
        df_updated,
        key_col='분류키',
        label_col='대분류',
        min_support=2,
        purity=0.7
    )

    fill_map = df_filled.dropna(subset=['대분류']).groupby('분류키')['대분류'].first()
    mask = df_updated['대분류'].isna()
    df_updated.loc[mask, '대분류'] = df_updated.loc[mask, '분류키'].map(fill_map)
    after = df_updated['대분류'].isna().sum()
    print(f"   대분류: {before:,} → {after:,} missing ({before-after:,} filled)")

    # Step 7: Clean up columns
    print("\n[Step 7] Cleaning up columns...")
    columns_to_drop = [
        '구분', '매출처코드', '우편번호', '옵션코드',
        '규격', '입수', '상품바코드', '대분류'
    ]

    # Only drop columns that exist
    columns_to_drop = [col for col in columns_to_drop if col in df_updated.columns]
    df_updated = df_updated.drop(columns_to_drop, axis=1)
    print(f"   Dropped {len(columns_to_drop)} columns")
    print(f"   Remaining columns: {list(df_updated.columns)}")

    # Step 8: Final cleanup
    print("\n[Step 8] Final cleanup...")
    df_updated = df_updated.reset_index(drop=True)

    # Summary statistics
    print("\n" + "=" * 80)
    print("PREPROCESSING COMPLETE")
    print("=" * 80)
    print(f"\nFinal dataset shape: {df_updated.shape}")
    print(f"Rows: {len(df_updated):,}")
    print(f"Columns: {len(df_updated.columns)}")
    print(f"\nMissing values per column:")
    print(df_updated.isnull().sum())
    print(f"\nUnique values per classification:")
    print(f"  중분류: {df_updated['중분류'].nunique()}")
    print(f"  소분류: {df_updated['소분류'].nunique()}")

    return df_updated


# ============================================================================
# SECTION 6: USAGE EXAMPLE
# ============================================================================

if __name__ == "__main__":

    # ========================================================================
    # Process PURCHASE data
    # ========================================================================
    print("\n" + "🔹" * 40)
    print("PROCESSING B물류센터 PURCHASE DATA")
    print("🔹" * 40)

    b_purchase_processed = preprocess_purchase_data(
        'B_purchase_data.csv',
        target_categories=['탄산음료', '봉지라면']
    )

    # Save processed purchase data
    # purchase_output = 'B_purchase_data_processed.csv'
    # b_purchase_processed.to_csv(purchase_output, index=False, encoding='utf-8-sig')
    # print(f"\n✅ Processed purchase data saved to: {purchase_output}")

    print("\nSample of processed purchase data:")
    print(b_purchase_processed.head(10))

    # ========================================================================
    # Process SALES data
    # ========================================================================
    print("\n\n" + "🔹" * 40)
    print("PROCESSING B물류센터 SALES DATA")
    print("🔹" * 40)

    b_sales_processed = preprocess_sales_data(
        'b_sales_2123.csv',
        'b_sales_24.csv',
        target_categories=['탄산음료', '봉지라면']
    )

    # Save processed sales data
    # sales_output = 'B_sales_data_processed.csv'
    # b_sales_processed.to_csv(sales_output, index=False, encoding='utf-8-sig')
    # print(f"\n✅ Processed sales data saved to: {sales_output}")

    print("\nSample of processed sales data:")
    print(b_sales_processed.head(10))

    # ========================================================================
    # Final Summary
    # ========================================================================
    print("\n\n" + "=" * 80)
    print("🎉 ALL B물류센터 PREPROCESSING COMPLETE!")
    print("=" * 80)
    print(f"\n📊 Purchase Data: {len(b_purchase_processed):,} rows")
    print(f"📊 Sales Data: {len(b_sales_processed):,} rows")
    print(f"\n💾 Output files:")
    # print(f"   - {purchase_output}")
    # print(f"   - {sales_output}")
    print(f"\n🎯 Filtered by 소분류: ['탄산음료', '봉지라면']")


🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹
PROCESSING B물류센터 PURCHASE DATA
🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹
B물류센터 PURCHASE DATA PREPROCESSING PIPELINE

[Step 1] Loading data...
   Initial rows: 345,021

[Step 2] Removing return entries (반출)...
   Rows after removal: 303,107

[Step 3] Creating classification keys...
   Unique keys created: 9,505

[Step 4] Initial classification filling (groupby method)...
   대분류: 5 → 2 missing (3 filled)
   중분류: 202,166 → 199,896 missing (2,270 filled)
   소분류: 205,486 → 203,531 missing (1,955 filled)

[Step 5] Filling missing barcodes...
   Barcodes: 72 → 64 missing (8 filled)

[Step 6] Filtering by target 소분류: ['탄산음료', '봉지라면']...
   Rows after filtering: 4,367
   Unique products: 202

[Step 7] Advanced classification filling...
   대분류: 0 → 0 missing (0 filled)

[Step 8] Cleaning up columns...
   Dropped 14 columns
   Remaining columns: ['일자', '공급업체 코드', '공급업체 우편번호', '상품명', 'EA', '중분류', '소분류', '분류키']

[Step 9] Final cleanup...

PREPROCESSING COMPL

In [26]:
categories_unique = b_purchase_processed['중분류'].unique()
print(categories_unique)

categories_unique = b_sales_processed['중분류'].unique()
print(categories_unique)

['라면류' '음료']
['라면류' '음료']


In [27]:
print('B_purchase')
print(b_purchase_processed.info())
print('<===================================================>')
print('B_sales')
print(b_sales_processed.info())

B_purchase
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4367 entries, 0 to 4366
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   일자         4367 non-null   object
 1   공급업체 코드    4367 non-null   object
 2   공급업체 우편번호  4367 non-null   object
 3   상품명        4367 non-null   object
 4   EA         4367 non-null   object
 5   중분류        4367 non-null   object
 6   소분류        4367 non-null   object
 7   분류키        4367 non-null   object
dtypes: object(8)
memory usage: 273.1+ KB
None
<===================================================>
B_sales
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72789 entries, 0 to 72788
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   판매일     72789 non-null  object
 1   판매수량    72789 non-null  object
 2   상품명     72789 non-null  object
 3   중분류     72789 non-null  object
 4   소분류     72789 non-null  object
 5   공급가액    72789 non-n